In [52]:
import intake
import os
import xarray as xr

# Define project-relative paths (adjusted since we're in notebooks/)
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
RAW_DATA_DIR = os.path.join(BASE_DIR, "data", "raw")
os.makedirs(RAW_DATA_DIR, exist_ok=True)  # make sure it exists


In [53]:
# Load CMIP6 cloud catalog (hosted on Google Cloud)
catalog_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(catalog_url)


In [54]:
query = col.search(
    source_id='GFDL-ESM4',
    experiment_id='historical',
    table_id='Amon',
    variable_id='tas',
    member_id='r1i1p1f1',
    grid_label='gr1'
)

# Load dataset (will trigger download from cloud storage)
dsets = query.to_dataset_dict()
ds = list(dsets.values())[0]
ds



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


<xarray.Dataset> Size: 411MB
Dimensions:         (bnds: 2, lat: 180, lon: 288, member_id: 1,
                     dcpp_init_year: 1, time: 1980)
Coordinates:
  * bnds            (bnds) float64 16B 1.0 2.0
    height          float64 8B ...
  * lat             (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
    lat_bnds        (lat, bnds) float64 3kB dask.array<chunksize=(180, 2), meta=np.ndarray>
  * lon             (lon) float64 2kB 0.625 1.875 3.125 ... 356.9 358.1 359.4
    lon_bnds        (lon, bnds) float64 5kB dask.array<chunksize=(288, 2), meta=np.ndarray>
  * time            (time) object 16kB 1850-01-16 12:00:00 ... 2014-12-16 12:...
    time_bnds       (time, bnds) object 32kB dask.array<chunksize=(1980, 2), meta=np.ndarray>
  * member_id       (member_id) object 8B 'r1i1p1f1'
  * dcpp_init_year  (dcpp_init_year) float64 8B nan
Data variables:
    tas             (member_id, dcpp_init_year, time, lat, lon) float32 411MB dask.array<chunksize=(1, 1, 600, 180, 288), meta=np.ndarray>
Attributes: (12/62)
    Conventions:                      CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            36500.0
    comment:                          <null ref>
    ...                               ...
    intake_esm_attrs:variable_id:     tas
    intake_esm_attrs:grid_label:      gr1
    intake_esm_attrs:zstore:          gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ES...
    intake_esm_attrs:version:         20190726
    intake_esm_attrs:_data_format_:   zarr
    intake_esm_dataset_key:           CMIP.NOAA-GFDL.GFDL-ESM4.historical.Amo...

In [55]:
ds = ds.squeeze()  # removes dimensions of size 1


# Subset: years 1940–2014, US-ish region
ds_subset = ds.sel(
    lat=slice(25, 50),            # south to north
    lon=slice(230, 300),          # west to east, 0–360 system
    time=slice("1940-01-01", "2014-12-31")
)
"""
ds_subset = ds_subset.where(
    (ds_subset.lat <= 50) & (ds_subset.lat >= 25) & # North to south
    (ds_subset.lon >= 230) & (ds_subset.lon <= 300), #west to east, adjusted to 0–360 system
    drop=True
)
"""

'\nds_subset = ds_subset.where(\n    (ds_subset.lat <= 50) & (ds_subset.lat >= 25) & # North to south\n    (ds_subset.lon >= 230) & (ds_subset.lon <= 300), #west to east, adjusted to 0–360 system\n    drop=True\n)\n'

In [56]:
print(ds['lat'].values[:10])  # first few lats
print(ds['lat'].values[-10:]) # last few


[-89.5 -88.5 -87.5 -86.5 -85.5 -84.5 -83.5 -82.5 -81.5 -80.5]
[80.5 81.5 82.5 83.5 84.5 85.5 86.5 87.5 88.5 89.5]


In [57]:
print("Num timesteps:", len(ds.time))

# Show the first few dates
print("First 5 timestamps:", ds.time.values[:5])

# Show the last few dates
print("Last 5 timestamps:", ds.time.values[-5:])

# Check the difference between first and second timestamp
print("Delta between first two:", ds.time.values[1] - ds.time.values[0])


Num timesteps: 1980
First 5 timestamps: [cftime.DatetimeNoLeap(1850, 1, 16, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(1850, 2, 15, 0, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(1850, 3, 16, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(1850, 4, 16, 0, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(1850, 5, 16, 12, 0, 0, 0, has_year_zero=True)]
Last 5 timestamps: [cftime.DatetimeNoLeap(2014, 8, 16, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(2014, 9, 16, 0, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(2014, 10, 16, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(2014, 11, 16, 0, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(2014, 12, 16, 12, 0, 0, 0, has_year_zero=True)]
Delta between first two: 29 days, 12:00:00


In [58]:
save_path = os.path.join(RAW_DATA_DIR, "gfdl_esm4_tas_1940_2014_us.nc")
ds_subset.to_netcdf(save_path)
print(f"Saved to {save_path}")


Saved to /Users/kyrakraft/Desktop/projects/climate-model-correction/data/raw/gfdl_esm4_tas_1940_2014_us.nc
